In [1]:
#Imports
import pandas as pd
import geopandas
import numpy as np
%matplotlib inline

import matplotlib.pyplot as plt

In [33]:
#Upload Data
signs = pd.read_csv("StreetSigns.csv")

# Displays sign counts of each unique neighbood
usigns = signs['neighborhood'].value_counts()
print(usigns)



South Side Flats             3672
Bloomfield                   3627
Central Business District    3491
Shadyside                    3370
Squirrel Hill South          3281
Brookline                    2256
Carrick                      2044
Central Lawrenceville        1970
Squirrel Hill North          1895
East Liberty                 1759
North Oakland                1743
Mount Washington             1630
Highland Park                1490
Beechview                    1397
Strip District               1366
Central Oakland              1327
Hazelwood                    1300
Point Breeze                 1284
Central Northside            1251
South Side Slopes            1175
East Allegheny               1066
North Shore                  1053
South Oakland                1049
Greenfield                   1048
Lower Lawrenceville          1024
Homewood South                999
Bluff                         978
Brighton Heights              903
Sheraden                      896
Garfield      

In [ ]:
# Creates a geopandas map of the data of street signs
signs = geopandas.read_file()